In [72]:
import os

import cv2 as cv
import numpy as np
from ultralytics import YOLO

model_path = os.path.join(
    os.path.dirname(os.getcwd()), "YOLO_model", "weights", "best.pt"
)
model = YOLO(model_path)

In [73]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [74]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape
    print(labels)
    print(res.boxes.conf)
    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [84]:
img = cv.imread("../scr.png")[:, :, :3]
img = cv.resize(img, (640, 640))
res = find_pieces(img)
from analysis import get_fen

print(get_fen(res, "w"))
print(res)


0: 640x640 1 black-king, 1 black-knight, 6 black-pawns, 1 black-queen, 1 white-bishop, 1 white-king, 1 white-knight, 7 white-pawns, 1 white-queen, 178.2ms
Speed: 2.5ms preprocess, 178.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


['white-bishop', 'white-knight', 'black-pawn', 'black-pawn', 'black-pawn', 'black-queen', 'black-pawn', 'white-pawn', 'white-queen', 'black-pawn', 'white-pawn', 'white-king', 'white-pawn', 'black-pawn', 'white-pawn', 'black-knight', 'black-king', 'white-pawn', 'white-pawn', 'white-pawn']
tensor([0.9758, 0.9448, 0.9290, 0.9215, 0.9126, 0.9115, 0.9115, 0.9041, 0.8931, 0.8889, 0.8807, 0.8792, 0.8721, 0.8490, 0.8433, 0.8428, 0.8141, 0.7793, 0.7456, 0.7428])
K4Q2/1PP5/P2PB1PP/4P3/4p2N/2npq1p1/ppp5/1k6 w
[['K' '' '' '' '' 'Q' '' '']
 ['' 'P' 'P' '' '' '' '' '']
 ['P' '' '' 'P' 'B' '' 'P' 'P']
 ['' '' '' '' 'P' '' '' '']
 ['' '' '' '' 'p' '' '' 'N']
 ['' '' 'n' 'p' 'q' '' 'p' '']
 ['p' 'p' 'p' '' '' '' '' '']
 ['' 'k' '' '' '' '' '' '']]
